## Session 02: Decision Trees

**From Wikipedia**

_Decision tree learning uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). It is one of the predictive modelling approaches used in statistics, data mining and machine learning. Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees._

We first define a very basic interface for a decision tree.

```
class DecisionTree(object):
    def __init__(self, split_scorer):
        ...

    def fit(self, data):
        ...

    def predict(self, row):
        ...
```

The `split_scorer` will be a callback which will allow us to determine which feature to split on next.

Now that we have a basic API defined, we can write a few scorer functions